# Module 3: Neural Network Practice

## Introduction
In this notebook, we will predict the status of a machine (working or failing) based on temperature and vibration sensor readings. This notebook builds on theoretical concepts from the course, including:

- Neural network architecture  
- Activation functions  
- Layers and neurons  
- Training via backpropagation  

We use TensorFlow in this notebook because it simplifies the process of building and training neural networks, making it easier for beginners to focus on core concepts like layers, activation functions, and training. With its high-level Keras API, TensorFlow handles complex operations like backpropagation and gradient updates for us, allowing for readable, intuitive code.


Nesta celda importamos as librarías necesarias para todo o cuaderno:

- **tensorflow**: Para construír e adestrar redes neuronais.
- **numpy**: Para manipulación de arrays e cálculos numéricos.
- **matplotlib.pyplot**: Para xeración de gráficas e visualización de datos.
- **sklearn**: Para partición de datos e escalado con `train_test_split` e `StandardScaler`.
- **ipywidgets** e **IPython.display**: Para crear unha interface interactiva con sliders.

Isto configura o entorno de traballo para as seguintes seccións.

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import ipywidgets as widgets
from IPython.display import display, clear_output

print("Libraries imported successfully!")

## Step 2: Load & Generate Data

Nesta celda cargamos o conxunto de datos dunha máquina que contén características de temperatura e vibración:

1. Usamos **pandas** para ler o arquivo CSV `machine_status.csv`.
2. Separamos as columnas de entrada (`temperature`, `vibration`) en `X` e a etiqueta binaria (`status`) en `y`.
3. Mostramos as primeiras filas para verificar a carga dos datos.

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv('./data/machine_status.csv')
X = df[['temperature', 'vibration']].values
y = df['status'].values

# Show samples
df.head()

## Step 3: Visualize the Data

Nesta celda visualizamos os datos nun diagrama de dispersión:

- Creamos unha figura de tamaño 10×6.
- Trazamos máquinas operativas (y == 0) e máquinas con fallo (y == 1) en cores diferentes.
- Engadimos título, etiquetas nos eixes, lenda e grade.

Isto axuda a inspeccionar visualmente a distribución dos datos.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', label='Working Machines')
plt.scatter(X[y==1, 0], X[y==1, 1], c='red', label='Failing Machines')
plt.title("Temperature vs. Vibration")
plt.xlabel("Temperature (°C)")
plt.ylabel("Vibration (m/s²)")
plt.legend()
plt.grid(True)
plt.show()

## Step 4: Prepare the Data

Nesta celda preparamos os datos para o adestramento:

1. Dividimos (`train_test_split`) os datos en conxuntos de adestramento e test (80%/20%).
2. Creamos un `StandardScaler` e:
   - Axustámolo aos datos de adestramento e transformamos `X_train`.
   - Transformamos `X_test` co mesmo scaler.

Normalizar os datos mellora o adestramento do modelo.

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Normalize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data prepared for training.")

## Step 5: Build the Neural Network Model

Nesta celda definimos a arquitectura da rede neuronal:

- Usamos un modelo **Sequential** de Keras.
- A primeira capa oculta ten 16 neuronas con activación **ReLU**.
- A capa de saída ten 1 neurona con activación **Sigmoid** para clasificación binaria.
- Compilamos co optimizador **Adam**, perda **binary_crossentropy** e métrica **accuracy**.

In [ ]:
model = Sequential([
    Input(shape=(2,)),
    Dense(16, activation='relu', input_dim=2),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print("Model created.")

## Step 6: Train the Model

Nesta celda adestramos o modelo:

- Chamamos a `model.fit` con:
  - Datos de adestramento escalados (`X_train_scaled`, `y_train`).
  - 50 épocas e tamaño de batch 32.
  - Validación do 20% dos datos.

Isto devolve un obxecto `History` co progreso do adestramento.

In [ ]:
history = model.fit(
    X_train_scaled, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)
print("Training complete.")

## Step 7: Evaluate the Model

Nesta celda avaliamos o modelo nos datos de proba:

- Usamos `model.evaluate` con `X_test_scaled` e `y_test`.
- Obtemos a perda e a precisión.
- Imprimimos estes valores para cuantificar o rendemento.

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

## Step 8: Interactive Prediction

Nesta celda creamos unha interface interactiva para predicións:

1. Definimos a función `predict_status(temp, vib)` que:
   - Forma un array cos valores de temperatura e vibración.
   - Aplica o scaler e obtén a predición do modelo.
   - Devolve 'Failing' ou 'Working' segundo o limiar 0.5.
2. Xera dous sliders (FloatSlider) para introducir valores e amosa o resultado dinámicamente.

In [ ]:
def predict_status(temp, vib):
    input_data = np.array([[temp, vib]])
    input_scaled = scaler.transform(input_data)
    prediction = model.predict(input_scaled)[0][0]
    return "Failing" if prediction > 0.5 else "Working"

def create_interface():
    temp_slider = widgets.FloatSlider(value=50, min=30, max=90, step=0.5, description='Temperature (°C):')
    vib_slider = widgets.FloatSlider(value=20, min=10, max=60, step=0.5, description='Vibration (m/s²):')
    output = widgets.Output()

    def update(change):
        with output:
            clear_output()
            status = predict_status(temp_slider.value, vib_slider.value)
            print(f"Temperature: {temp_slider.value}°C")
            print(f"Vibration: {vib_slider.value} m/s²")
            print(f"Prediction: {status}")

    temp_slider.observe(update, names='value')
    vib_slider.observe(update, names='value')
    display(widgets.VBox([temp_slider, vib_slider, output]))

create_interface()